In [ ]:
### User-friendly recoil-frame and radial-frame covariance code ###

# This code is intended to contain the radial-frame covariance adaptation of the pre-existing recoil-frame covariance code,
# where the code is intended to run on either simulated or real PImMS data (specified by user input). Below is a draft of how
# each section is intended to run:

# 1: Functions and imports, and defining the 'beautify' function (for making graphs look pretty) and the 'create_image' function
#    for producing centred ion-images in a given time-interval.

# 2. Define pre-existing recoil-frame covariance function with radial-frame adaptation

# 3. Loading data (according to whether user wants simulated or real data to be used):
#     - If simulated data, load the _pixelated.csv file directly and save as df.
#     - If real data, access .bin file and convert to df (using procedure in "on the fly ion images..." Python script).

# 4. Plotting TOF spectrum (possible adaptation for run-averaged TOF spectrum for real-data).

# 5. Calibration of TOF spectrum to convert from TOF-domain to m/z domain (user input for predicted m/z values and corresponding
#    TOF values from previous TOF spectrum assignment) via linear-regression. Add extra column to df for m/z values using linear
#    regression parameters obtained, and plot m/z-domain spectrum.

# 6. Plot raw ion-images for specified ions of choice (supplied as user input). May need to change centre values for certain
#    ions.

# 7. Perform Abel-inversion on raw (centred) ion-images, and obtain radial disributions of ions.

# 8. Compress data

# 9. Calculating ion covariances, calling the main covariance function defined in (2).

# 10. Plotting the recoil-frame covariance maps.

# 11. Plotting the radial-frame covariance maps.

# 12. Plotting final radial-frame covariance map with radial distribution functions on axes.

# 13. Scaling the radial covariance matrices relative to the maximum matrix entry (across all radial matrices) and re-plotting.

In [ ]:
# 1: Functions and imports, and defining the beautify function (for making graphs look pretty) and the create_image function
#    for producing centred ion-images in a given time-interval.
#    Adapted from: "Recoil-frame covariance (for Part IIs, 220930).ipynb"

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
import os
import matplotlib.patches as patches
import multiprocessing as mp
from scipy import ndimage
from scipy.ndimage import gaussian_filter
import gc # garbage collector - helps with data storage 

# Function to make images look pretty
def beautify(ax):
    ax.tick_params(axis = 'x', labelsize = 10)
    ax.tick_params(axis = 'y', labelsize = 10)
    ax.tick_params('both', length = 3, width = 1)
    ax.spines['bottom'].set_linewidth(1)
    ax.spines['top'].set_linewidth(1)
    ax.spines['right'].set_linewidth(1)
    ax.spines['left'].set_linewidth(1)

# Function for producing centred ion-images within a specified time-interval and a specified ion-image centre
def create_image(df, centre, ti = 0, tf = 0):
    if (ti & tf):
        df = df[(df['ToF'] >= ti) & (df['ToF'] <= tf)] # sets time-interval for image (replace ToF with t for sim. data)
    image = np.zeros((324, 324)) # sets initial array for image
    xy = df.groupby(['x', 'y']).size().reset_index(name = 'intensity') # computes the intensity for each xy coordinate
    results = (np.array(xy).astype(int))
    cx = 162 - centre[0] # shift in x axis for centred image (by manually defined centre in function input)
    cy = 162 - centre[1] # shift in y axis for centred image (by manually defined centre in function input)
    for a,b,c in results: # assigns the intensity (c) to each coordinate in the image (from specified results input)
        if 0 <= (b + cx) < 324:
            if 0 <= (a + cy) < 324:
                image[b + cx, a + cy] = c
        
    return(image) # returns centred image

In [ ]:
# 2. Define pre-existing recoil-frame covariance function with radial-frame adaptation.
#    Adapted from: "Recoil-frame covariance (for Part IIs, 220930).ipynb"

from numba import jit 

@jit(nopython = True) # think this is just a way to make the calculations run quicker / in parallel(?) once function is called
def recoil_frame_Sij_2(array_A, array_B, shot_array, pixels, mean_A, mean_B): # for norm; array_A and array_B are df_A and df_B
    
    Sij_array = np.zeros((pixels, pixels), dtype = np.float64) # setting up 2D recoil-frame covariance array
    radial_Sij_array = np.zeros((int(pixels/2), int(pixels/2))) # setting up 2D radial-radial covariance array

   
    no_A = array_A.shape[0] # array_A = df_A; shape returns no of shots with A in (+ dummy shot)
    no_B = array_B.shape[0] # array_B = df_B; shape returns no of shots with B in (+ dummy shot)
#     print("array a recoil_frame_Sij_2")
#     print(array_A) # r, theta, shot
#     print("array b recoil_frame_Sij_2")
#     print(array_B) # r, theta, shot
    #print("recoil_frame_Sij_2 (no_A no_B)")
    #print(no_A, no_B)
    
    nshotsA, nshotsB = 0, 0
    nendA, nendB = 0, 0
#     print(shot_array)
    for shot in shot_array: # loop over all shots in shot array
        # print('Shot:'+str(shot))
        A_in_shot, B_in_shot = False, False
        
        for test in range(nshotsA, no_A): # looping over shots with A in them; marking whether A is in shot or not
            if (array_A[test, 2] == shot):
                nshotsA = test
#                 print('A Shots:'+str(nshotsA))
                A_in_shot = True
                break
        
        if A_in_shot:
            for test in range(nshotsA, no_A):
                if array_A[test, 2] > shot:
                    nendA = test
#                     print('A End:'+str(nendA))
                    break

            for test in range(nshotsB, no_B):
                if (array_B[test, 2] == shot):
                    nshotsB = test
#                     print('B Shots:'+str(nshotsB))
                    B_in_shot = True
                    break
        
        if B_in_shot:
            for test in range(nshotsB, no_B):
                if array_B[test, 2] > shot:
                    nendB = test
#                     print('B End:'+str(nendB))
                    break
        
        if A_in_shot & B_in_shot:
#             print('A and B found')
#             print(nendA, nshotsA)
            n_A_ions = (nendA - nshotsA)
            n_B_ions = (nendB - nshotsB)
#             print('A ions:', n_A_ions)
#             print('B ions:', n_B_ions)
            for i in range(nshotsA, nendA):
                rad_A = array_A[i, 0]
                theta_A = array_A[i, 1]
#                 print(rad_A, theta_A)

                for j in range(nshotsB, nendB):
                    rad_B = array_B[j, 0]
                    theta_B = array_B[j, 1]
                
                    # need to convert radii to integers
                    rad_A_int = int(round(rad_A))
                    rad_B_int = int(round(rad_B))
                    #print(rad_A_int, rad_B_int)

                    theta_rel = theta_B - theta_A # relative angle between two hits
                    
                    if (0 < rad_A_int < pixels/2) and (0 < rad_B_int < pixels/2): # if hit is within image array
                        radial_Sij_array[rad_A_int, rad_B_int] += mean_A/n_A_ions
                    
                    # correction to make sure all relative angles are between 0 and 2*pi
                    if theta_rel>2*np.pi:
                        theta_rel=theta_rel-2*np.pi
                    elif theta_rel<0:
                        theta_rel=theta_rel+2*np.pi

                    x = int(round(rad_B*np.cos(theta_rel) + pixels/2))
                    y = int(round(rad_B*np.sin(theta_rel) + pixels/2))
#                     print(x,y)

                    if (0 < x < pixels) and (0 < y < pixels): # if hit is within the image array
                        Sij_array[y-1, x-1] += mean_A/n_A_ions
#                         Sij_array[y-1, x-1] += mean_B
#                         Sij_array[y-1, x-1] += mean_B/n_B_ions
#                         Sij_array[y-1, x-1] += int(1)/n_A_ions
#                         Sij_array[y-1, x-1] += int(1)
#                         print(mean_A,n_A_ions)
#                         print('Sij_array:')
#                         print(np.unique(Sij_array))
    
    return(Sij_array, radial_Sij_array)

@jit(nopython = True)
def recoil_frame_SiSj(reduced_array_A, reduced_array_B, pixels): # for not norm; reduced_array_A = A_array_SiSj (and likewise for B)
    
    SiSj_array = np.zeros((pixels, pixels), dtype = np.int64)
    radial_SiSj_array = np.zeros((int(pixels/2), int(pixels/2)))
    
    no_A = reduced_array_A.shape[0]
    no_B = reduced_array_B.shape[0]
#     print(no_A)
#     print(no_B)
    
    for i in range(no_A): 
#         print(i)
        rad_A = reduced_array_A[i, 0]
        theta_A = reduced_array_A[i, 1]
        count_A = reduced_array_A[i, 2]

        for j in range(no_B):
#             print(i, j)
            rad_B = reduced_array_B[j, 0]
            theta_B = reduced_array_B[j, 1]
            count_B = reduced_array_B[j, 2]
            
#             if rad_A and rad_B:
            if 3:
        
                theta_rel = theta_B - theta_A
                if theta_rel > 2*np.pi:
                    theta_rel = theta_rel - 2*np.pi
                elif theta_rel < 0:
                    theta_rel = theta_rel + 2*np.pi

                x = int(round(rad_B*np.cos(theta_rel) + pixels/2))
                y = int(round(rad_B*np.sin(theta_rel) + pixels/2))
                
                # need to convert radii to integers
                rad_A_int = int(round(rad_A))
                rad_B_int = int(round(rad_B))
                
#                 print(x,y, theta_rel)
#                 print(count_A, count_B)
                if (0 < x < pixels) and (0 < y < pixels): # if hit is within the image array
                    SiSj_array[y-1, x-1] += int(count_A*count_B)
                if (0 < rad_A_int < pixels/2) and (0 < rad_B_int < pixels/2):
                    radial_SiSj_array[rad_A_int, rad_B_int] += int(count_A*count_B)
#                     print(np.unique(SiSj_array))
    
    return(SiSj_array, radial_SiSj_array)

def two_fold_recoil_frame_covariance(df_A, df_B, shot_list, pixels, norm = False): # MAIN COVARIANCE FUNCTION 

    n_shots = len(shot_list)-1 # number of shots (in this case, 10000)
#     print('Shot length:'+str(n_shots))
    
######## Sij = <xy> ################################################################################
    
    start_time = time.time() # just starts a timer
    
    A_array = df_A[['r', 'theta', 'tId']].values # r, theta, shot no array from df_A
    B_array = df_B[['r', 'theta', 'tId']].values # r, theta, shot no array from df_B
    
    #print("covariance A_array")
    #print(A_array)
    #print("covariance B_array")
    #print(B_array)
    
    shot_array_A = np.unique(df_A.tId) # numpy array of unique shot numbers containing A
    shot_array_B = np.unique(df_B.tId) # numpy array of unique shot numbers containing B
    
    shot_array = np.intersect1d(shot_array_A, shot_array_B) # array of shot numbers containing both ion A and ion B
    shot_array = shot_array.astype('int64')
    #print("shot_array covariance")
    #print(shot_array)

    A_array = np.vstack([A_array, [-1000,-1000,len(shot_list)]]) # Add dummy shots to fix counting issue
    B_array = np.vstack([B_array, [-1000,-1000,len(shot_list)]])
    #print("a array new")
    #print(A_array.shape)
    #print("b array new")
    #print(B_array.shape)    
    
    #print("len df_A")
    #print(len(df_A))
    #print("len df_B")
    #print(len(df_B))
    
    # what does norm mean? A: normalisation
    if norm:
        mean_A = (len(df_A))/(n_shots)   # likelihood of ion A appearing in a given shot
        mean_B = (len(df_B))/(n_shots)   # likelihood of ion B appearing in a given shot      
#         print('Length A: ', len(df_A))
#         print('Mean A: ', mean_A) # likelihood of ion A appearing in a given shot
#         print('Length B: ', len(df_B))
#         print('Mean B: ', mean_B) # likelihood of ion B appearing in a given shot
#         print(len(df_A)-1,n_shots,mean_A)
        Sij, Sij_rad = recoil_frame_Sij_2(A_array, B_array, shot_array, pixels, mean_A, mean_B) ### function 1 to look at
#         print("test test test for if norm")
        Sij = Sij.astype('float64')
        Sij_rad = Sij_rad.astype('float64')
#         print(np.unique(Sij))
        Sij /= float(n_shots) # divide by total number of shots
        Sij_rad /= float(n_shots)
#         print(np.unique(Sij), n_shots)
#         print(Sij)

    else: # currently not in use ***********
        mean_A = (len(df_A)-1)/n_shots
#         print('Mean A', mean_A)
#         print(A_array, B_array)
        Sij = recoil_frame_Sij_1(A_array, B_array, shot_array, pixels, mean_A) ### function 2 to look at 
        Sij = Sij.astype('float64')
#         print(np.unique(Sij))
        Sij /= float(n_shots)
#         print(np.unique(Sij), n_shots)
        
    print("Calculating Sij took %s seconds" % round((time.time() - start_time), 1)) # times how long it took to calculate Sij
    
########### SiSj = <x><y> ################################################################################
    
    start_time = time.time()
        
    df_A_count_series = df_A.groupby(['r', 'theta']).size()
#     print("df_A_count_series")
#     print(df_A_count_series)
#     print(df_A_count_series.shape)
    df_A_count_df_SiSj = df_A_count_series.to_frame(name = 'size').reset_index()
#     print("df_A_count_df_SiSj")
#     print(df_A_count_df_SiSj)
#     print(df_A_count_df_SiSj.shape)

    df_B_count_series = df_B.groupby(['r', 'theta']).size()
    df_B_count_df_SiSj = df_B_count_series.to_frame(name = 'size').reset_index()
#     print(df_B_count_df_SiSj)
    
    A_array_SiSj = df_A_count_df_SiSj[['r', 'theta', 'size']].values # just converts from df to numpy array
#     print("A_array_SiSj")
#     print(A_array_SiSj)
    B_array_SiSj = df_B_count_df_SiSj[['r', 'theta', 'size']].values # just converts from df to numpy array
#     print("B_array_SiSj")
#     print(B_array_SiSj)
    
#     print(A_array_SiSj, B_array_SiSj)    
    SiSj, SiSj_rad = recoil_frame_SiSj(A_array_SiSj, B_array_SiSj, pixels) # calculates <SiSj>
#     print(SiSj)
    SiSj = SiSj.astype('float64')
    SiSj_rad = SiSj_rad.astype('float64')
#     print(np.unique(SiSj))
    SiSj /= (n_shots)**2
    SiSj_rad /= (n_shots)**2
#     SiSj /= (n_shots-1)**2
#     print(np.unique(SiSj))
#     SiSj /= mean_A
    
    
    print("Calculating SiSj took %s seconds" % round((time.time()-start_time),1))
    
    return(Sij, SiSj, Sij_rad, SiSj_rad)
   

In [ ]:
# 3. Loading data (according to whether user wants simulated or real data to be used):
#     - If simulated data, load the _pixelated.csv file (produced from CEI simulation script) directly and save as df.
#     - If real data, access .bin file and convert to df (using procedure in "on the fly ion images..." Python script).
#    Adapted from: "on the fly ion images + delay + rdf NOT CENTROIDED v6 EMW C4H4Se.ipynb"

file_type = int(input("Choose whether you would like to analyse simulated data ('0') or real data ('1'): "))
    
# If wanting to analyse simulated data 
if file_type == 0:
    
    df_list = []
    
    df = str(input("Enter the _pixelated.csv file name: "))
    directory=os.chdir(r"/home/merrickj/Documents/simulated_data_figs")
    df = pd.read_csv(f"{df}", names=['x','y','ToF','tId'], usecols=[0,1,2,3], header=0)
    print(df)
    
    df_list.append(df)
    
# If wanting to analyse real data
elif file_type == 1:
    
    import pandas as pd
    import numpy as np
    import math
    import os
    import re
    import time
    from numpy import exp, loadtxt, pi, sqrt
    import matplotlib.cm as cm
    import gc
   
    pd.set_option('display.max_rows', 4096) # Controls number of lines displayed (PImMS maximum = 4096 timebins)
    
    #directory=os.chdir(r"/home/merrickj/Documents/indene_filtered_data_for_covariance")
    #directory=os.chdir(r"/home/merrickj/Documents/fluorene_filtered_data_for_covariance")
    #directory=os.chdir(r"/home/merrickj/Documents/CPP_filtered_data_for_covariance")
    directory=os.chdir(r"/home/merrickj/Documents/fluorene_2018_all")

    #file_list = ['038','039','040','041','042','043','044'] # run number; adapt to supply an array of run numbers whose data can be aggregated over
    #file_list = ['222','223','224','226','227'] # run numbers for indene (2023) beamtime data
    
    #file_list = ['indene_full_data_centroided_corrected']
    #file_list = ['fluorene_full_data_centroided_corrected']
    #file_list = ['CPP_full_data_centroided_corrected']
    file_list = ['fluorene_full_data_centroided_uncorrected']
    
    df_list = []
    
    for f in zip(file_list):
        print(f[0])
        print('Opening file: '+f[0])
        data = np.load(f"{f[0]}.npy", mmap_mode='r') # loads centroided and filtered numpy array
        #df = pd.DataFrame(data, columns = ['Energy','tId','x','y','ToF','size','spread','delay','m_z']) # for centroided
        df = pd.DataFrame(data, columns = ['x','y','ToF','tId','delay','size','spread']) # for centroided 2018
        print(df)
        df_list.append(df)
        
        del data
        gc.collect()
    
else:
    sys.exit('Invalid data-type supplied.')

In [ ]:
# 4a. Plotting run-separated TOF spectrum. This should work - if memory issues arise, just restart the kernel.
#    Adapted from: "on the fly ion images + delay + rdf NOT CENTROIDED v6 EMW C4H4Se.ipynb"

# Simulated data
if file_type == 0:
    fig, ax = plt.subplots(figsize=(15, 6))
    #beautify(ax)
    tof = df.groupby(['ToF']).size().reset_index(name='intensity')
    plt.bar(tof.ToF,tof.intensity, width = 0.1)
    plt.xlabel('Time-of-flight / a.u.', fontsize=16)
    plt.tick_params(axis='x', labelsize=12)
    plt.ylabel('Intensity / a.u.', fontsize=16)
    plt.tick_params(axis='y', labelsize=12)
    ax.grid(which='major')
    ax.grid(which='minor')
    ax.minorticks_on()
    plt.title('ToF spectrum - ethene (simulated CEI data)', fontsize = 16)
    ax.set_xlim(0,15)
    fig.savefig('ethene_tof_spectrum.png')
    plt.show()
    
# Real data - seperated by run number, and then summed over all run numbers
if file_type == 1:
    fig, ax = plt.subplots(figsize=(15, 6))
    beautify(ax)
    plt.xlabel('Time-of-flight / a.u.', fontsize=12)
    plt.tick_params(axis='x', labelsize=12)
    plt.ylabel('Intensity / a.u.', fontsize=12)
    plt.tick_params(axis='y', labelsize=12)
    ax.set_xlim(2050,2500) # alter as necessary
    ax.grid(which='major')
    ax.grid(which='minor')
    ax.minorticks_on()
    tof_appended = []
    for df,file in zip(df_list,file_list):
        
        #plt.title('Run-concatenated ToF spectrum - indene', fontsize = 16)
        #plt.title('Run-concatenated ToF spectrum - fluorene', fontsize = 16)
        #plt.title('Run-concatenated ToF spectrum - CPP', fontsize = 16)
        plt.title('Run-concatenated ToF spectrum - fluorene (2018)', fontsize = 16)
        
        #df = df[(df['ToF']>1550) & (df['ToF']<1900)]
        tof = df.groupby(['ToF']).size().reset_index(name='intensity')
        tof_times_list = tof['ToF'].to_list()
        #print(tof)
        tof_appended.append(tof)
        plt.plot(tof.ToF,tof.intensity,label=str(file), color = 'black')
        #ax.legend(loc='upper right', fontsize=18)
    plt.show()
    
    #directory=os.chdir(r"/home/merrickj/Documents/indene_ion_figs_and_cov_plots")
    #fig.savefig('indene_concatenated_tof_spectrum.png')
    
    #directory=os.chdir(r"/home/merrickj/Documents/fluorene_ion_figs_and_cov_plots")
    #fig.savefig('fluorene_concatenated_tof_spectrum.png')
    
    #directory=os.chdir(r"/home/merrickj/Documents/CPP_ion_figs_and_cov_plots")
    fig.savefig('fluorene_2018_concatenated_tof_spectrum.png')

In [ ]:
# 6a. Plot raw ion-images for specified ions of choice (supplied as user input). May need to change centre values for certain
#    ions.
#    Adapted from: "Recoil-frame covariance (for Part IIs, 220930).ipynb"

# for fluorene, 2018 data
# pixels = 324
# ion_list = ['C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12']
# ToF_list = [(2219,2221),(2249,2251),(2279,2281),(2304,2306),(2326,2328),(2349,2351),(2367,2369),(2392,2394),(2409,2411),(2425,2427),(2441,2443)]
# centre_list = [(160,135),(160,130),(161,129),(160,125),(160,120),(160,115),(160,110),(159,109),(160,105),(160,102),(160,99)] # centres of ion images - fine-tune as necessary

# for triphenylene, 2021 data
#pixels = 324
#ion_list = ['CH$^{2+}$','C$^{+}$','C$_{2}$H$_{2}^{+}$','C$_{3}$H$_{x}^{+}$','C$_{4}$H$_{x}^{+}$','C$_{5}$H$_{x}^{+}$','C$_{6}$H$_{x}^{+}$','C$_{7}$H$_{x}^{+}$','C$_{8}$H$_{x}^{+}$','C$_{9}$H$_{x}^{+}$']
#ToF_list = [(1044,1046),(1061,1063),(1097,1099),(1119,1121),(1139,1141),(1156,1158),(1172,1174),(1187,1189),(1202,1204),(1216,1218)]
#centre_list = [(165,162),(175,162),(178,162),(192,162),(195,162),(196,162),(198,162),(202,162),(205,162),(210,162)]

# for fluorene, 2023 data (wrong)
#pixels = 324
#ion_list = ['C12(+)','C11(+)','C10(+)','C9(+)','C8(+)','C7(+)','C6(+)','C11(2+)','C5(+)','C9(2+)','C4(+)','C11(3+)','C3(+)','C9(3+)','C2(+)']
#ToF_list = [(1824,1826),(1811,1813),(1800,1802),(1787,1789),(1772,1774),(1754,1756),(1737,1739),(1729,1731),(1711,1713),(1700,1702),(1691,1693),(1679,1681),(1671,1673),(1649,1651)]
#centre_list = [(222,140),(220,140),(215,140),(215,140),(212,140),(205,140),(201,140),(200,140),(200,140),(195,140),(195,140),(191,140),(190,140),(185,140),(180,140)]
#print(len(ion_list))
#print(len(ToF_list))
#print(len(centre_list))

# pixels = 324
# ToF_list = [(1613, 1616),(1617, 1621),(1631, 1634),(1643, 1646),(1650, 1652), (1657, 1662),(1662,1671),(1670, 1673), (1679,1683),(1688,1695),(1699,1705), (1708,1714), (1716,1723), (1725,1735),(1734,1741),(1743,1750), (1749,1755),(1756,1765),(1770,1776),(1783,1793), (1794,1803), (1810,1816),(1831,1841)]
# ion_list = ['N(++)','O(++)','C2(++)','O(+)','water(+)','carbon_dioxide(++)','C2(+)','dinitrogen(+)','dioxygen(+)','C3(+)','carbon_dioxide(+)','C4(+)','PAH(+++)','C5(+)','C11(++)','C6(+)','PAH(++)','C7(+)','C8(+)','C9(+)','C10(+)','C11(+)','PAH-H(+)']
# centre_list = [(162,162),(162,162),(182,140),(162,162),(162,162),(162,162),(188,140),(162,162),(162,162),(193,140),(162,162),(198,140),(203,140),(203,140),(203,140),(206,140),(206,140),(212,140),(215,140),(215,140),(217,140),(218,140),(225,140)]

# for indene, 2023 data
#ToF_list = [(1630,1634),(1665,1670),(1685,1697),(1700,1703),(1708,1718),(1720,1722),(1725,1735),(1740,1750),(1755,1765),(1785,1795)]
#ion_list = ['C1','C2','C3','C7(2+)','C4','IND(2+)','C5','C6','C7','IND(+)']
#centre_list = [(185,144),(190,144),(197,144),(199,144),(201,144),(202,144),(205,144),(208,144),(213,144),(217,144)] # change manually

# for fluorene, 2023 data
# ToF_list = [(1831,1840),(1820,1830),(1810,1818),(1798,1802),(1780,1792),(1770,1778),(1755,1765),(1750,1752),(1742,1749),(1735,1740),(1725,1733),(1720,1722),(1707,1715),(1699,1704),(1685,1698),(1662,1672),(1630,1633)]
# ion_list = ['FLU(+)','C12(+)','C11(+)','C10(+)','C9(+)','C8(+)','C7(+)','FLU(++)','C6(+)','C11(++)','C5(+)','FLU(+++)/C9(++)','C4(+)','C3(+)','C7(++)/FLU(++++)','C2(+)','C1(+)']
# centre_list = [(222,142),(221,142),(220,142),(217,142),(212,142),(210,142),(205,142),(205,142),(203,142),(201,142),(200,142),(198,142),(198,142),(195,142),(192,142),(189,142),(180,142)]

# for CPP, 2023 data
# ToF_list = [(1631,1636),(1665,1671),(1686,1694),(1701,1707),(1710,1715),(1718,1722),(1725,1733),(1736,1740),(1742,1749),(1752,1754),(1757,1765),(1766,1770),(1772,1778),(1780,1791),(1795,1805),(1806,1815),(1820,1828),(1830,1838),(1842,1850),(1851,1860),(1864,1869)]
# ion_list = ['C1','C2','C3','CPP(4+)','C4','C9(2+)','C5','C11(2+)','C6','C13(2+)','C7','CPP(2+)','C8','C9','C10','C11','C12','C13','C14','CPP(+)','[CPP-dinitrogen](+)']
# centre_list = [(191,145),(193,145),(195,145),(198,145),(203,145),(205,145),(205,145),(207,145),(210,145),(211,145),(213,145),(213,145),(215,145),(219,145),(222,145),(225,145),(227,145),(230,145),(231,145),(235,145),(238,145)]

# for ethene, simulated data
ToF_list = [(7,7),(14,14)]
ion_list = ['$CH_{2}^{++}$','$CH_{2}^{+}$']
centre_list = [(162,162),(162,162)]

if len(ion_list) == len(centre_list) == len(ToF_list):
    print("All arrays of same length.")
    
# Simulated data
if file_type == 0:
    
    list_of_total_images = []
    
    for i, (Ti, Tf) in enumerate(ToF_list):
        ion = ion_list[i]
        centre = centre_list[i]
        fig, ax = plt.subplots(figsize = (7, 7))
        image = create_image(df, centre, Ti, Tf)
        list_of_total_images.append(image)
        plt.ylabel('Pixels', fontsize=12, labelpad = 5)
        plt.xlabel('Pixels', fontsize=12, labelpad = 5)
        ax.imshow(image, interpolation='nearest',cmap = 'inferno', vmax = 0.5*np.max(image), vmin = 0, origin = 'lower')
        ax.set_xlim(0,324)
        ax.set_ylim(324,0) # Inverted to keep x,y directions the same as above
        ax.text(0.04, 0.88, ion, transform = ax.transAxes, fontsize = 30, color = 'white')
        ax.grid(which='major')
        ax.grid(which='minor', linewidth=0.3)
        ax.minorticks_on()
        fig.savefig(f"ion_image_{i}_ethene.png")

# Real data
if file_type == 1:
    
    #directory=os.chdir(r"/home/merrickj/Documents/indene_filtered_data_for_covariance")
    #directory=os.chdir(r"/home/merrickj/Documents/fluorene_filtered_data_for_covariance")
    #directory=os.chdir(r"/home/merrickj/Documents/CPP_filtered_data_for_covariance")
    
    list_of_total_images = [] # for saving centroided data
    
    single_image = input("Do you wish to generate only single ion-images (y/n)?")
    
    for i, (Ti, Tf) in enumerate(ToF_list):
        image_store = []
        ion = ion_list[i]
        centre = centre_list[i]
        
        for df,file in zip(df_list,file_list):
            
            print('Opening file: '+str(file))
            
            if single_image in ['y', 'Y', 'yes', 'Yes', 'YES']: # if wanting to generate only single ion-images
                
                first_tId = df['tId'].iloc[0] # for plotting ion image just for one BID/tId to verify centroiding conditions
                df_test = df[(df['tId'] == first_tId)]
                print(f"First tId for centroided data: {first_tId}")
                image = create_image(df_test, centre, Ti, Tf)
                image_store.append(image)
            
            else:
                
                image = create_image(df, centre, Ti, Tf)
                image_store.append(image)
            
        total_image = sum(image_store)
        print('Finished creating ion-image for ion: '+ ion)
        list_of_total_images.append(total_image)
        
    print("*** Centroided ion-images now fully generated. ***")
    
    uncentroided_comparison = input("Do you wish to generate ion-images for uncentroided data (y/n)?")
    
    if uncentroided_comparison in ['y', 'Y', 'yes', 'Yes', 'YES']: # if wanting to generate ion-images for uncentroided data
        
        #uncentroided_data = np.load("indene_full_data_uncentroided_corrected.npy",allow_pickle=True)
        #uncentroided_data = np.load("fluorene_full_data_uncentroided_corrected.npy",allow_pickle=True)
        uncentroided_data = np.load("CPP_full_data_uncentroided_corrected.npy",allow_pickle=True)
        
        uncentroided_data_df = pd.DataFrame(uncentroided_data, columns=['Energy','tId','x','y','ToF','counter_tagID','delay'])
        print("Uncentroided concatenated dataframe:")
        print(uncentroided_data_df)
        
        list_of_total_images_uncentroided = []
        
        for i, (Ti, Tf) in enumerate(ToF_list):
            
            image_store = []
            ion = ion_list[i]
            centre = centre_list[i]
            
            if single_image in ['y', 'Y', 'yes', 'Yes', 'YES']: # if wanting to generate only single ion-images

                first_tId = uncentroided_data_df['tId'].iloc[0] # for plotting ion image just for one BID/tId to verify centroiding conditions
                print(f"First tId for non-centroided data: {first_tId}") # just to check that non-centroided and centroided plots involve the same tId's worth of data
                df_test = df[(uncentroided_data_df['tId'] == first_tId)]
                image = create_image(df_test, centre, Ti, Tf)
                image_store.append(image)
                
            else:
                
                image = create_image(uncentroided_data_df, centre, Ti, Tf)
                image_store.append(image)

            total_image = sum(image_store)
            print('Finished creating ion-image for ion: '+ ion)
            list_of_total_images_uncentroided.append(total_image)
            
        print("*** Non-centroided ion-images now also fully generated. ***")
        
    else: # if not wanting to look at uncentroided data for comparison
        
        pass

In [ ]:
# 6b. Plot raw ion-images for specified ions of choice (supplied as user input). May need to change centre values for certain
#    ions. This is a separate plotting function to the above to avoid having to run the create_image function several times
#    (takes time for several fragments!) to allow for easy image editing once images are generated.
#    Adapted from: "Recoil-frame covariance (for Part IIs, 220930).ipynb"

#directory=os.chdir(r"/home/merrickj/Documents/indene_ion_figs_and_cov_plots")
#directory=os.chdir(r"/home/merrickj/Documents/fluorene_ion_figs_and_cov_plots")
#directory=os.chdir(r"/home/merrickj/Documents/CPP_ion_figs_and_cov_plots")

crosshairs = False # if wanting to plot crosshairs to help with centring (taken from FLASH_compilation_2023_modified script)

for i, (Ti, Tf) in enumerate(ToF_list):
    ion = ion_list[i]
    
    total_image = list_of_total_images[i]
    
    if uncentroided_comparison in ['y', 'Y', 'yes', 'Yes', 'YES']: # if wanting to plot ion-images for uncentroided data; note that crosshairs aren't an option in this case as not needed
        
        total_image_uncentroided = list_of_total_images_uncentroided[i]
        
        fig, (ax1,ax2) = plt.subplots(nrows = 1, ncols = 2, figsize = (18,8)) # for plotting centroided and uncentroided side by side
        
        ax1.imshow(total_image, interpolation='nearest',cmap = 'inferno', vmax = 0.0008*np.max(total_image), vmin = 0, origin = 'lower')
        ax1.set_xlim(0,324)
        ax1.set_ylim(324,0) # Inverted to keep x,y directions the same as above
        ax1.set_ylabel('Pixels', fontsize=12, labelpad = 5)
        ax1.set_xlabel('Pixels', fontsize=12, labelpad = 5)
        ax1.grid(which='major')
        ax1.grid(which='minor', linewidth=0.3)
        ax1.minorticks_on()
        
        ax2.imshow(total_image_uncentroided, interpolation='nearest',cmap = 'inferno', vmax = 0.0008*np.max(total_image_uncentroided), vmin = 0, origin = 'lower') # massively oversaturated to see ion hits more clearly in comparison
        ax2.set_xlim(0,324)
        ax2.set_ylim(324,0) # Inverted to keep x,y directions the same as above
        ax2.set_ylabel('Pixels', fontsize=12, labelpad = 5)
        ax2.set_xlabel('Pixels', fontsize=12, labelpad = 5)
        ax2.grid(which='major')
        ax2.grid(which='minor', linewidth=0.3)
        ax2.minorticks_on()
        
        if single_image in ['y', 'Y', 'yes', 'Yes', 'YES']: # if wanting to generate only single ion-images
            
            ax1.set_title(f"Single-shot ion-image for ion: {ion} (centroided)", fontsize = 14, loc = 'center')
            ax2.set_title(f"Single-shot ion-image for ion: {ion} (non-centroided)", fontsize = 14, loc = 'center')
            
            fig.savefig(f"ion_image_comparison_{i}_single.png")
            
        else:
            
            ax1.set_title(f"Total ion-image for ion: {ion} (centroided)", fontsize = 14, loc = 'center')
            ax2.set_title(f"Total ion-image for ion: {ion} (non-centroided)", fontsize = 14, loc = 'center')           
        
    else: # if not wanting to plot ion-images for uncentroided data
        
        fig, ax = plt.subplots(figsize = (8, 8))
        ax.imshow(total_image, interpolation='nearest',cmap = 'inferno', vmax = 0.6*np.max(total_image), vmin = 0, origin = 'lower')
        ax.set_xlim(0,324)
        ax.set_ylim(324,0) # Inverted to keep x,y directions the same as above
        #ax.text(0.04, 0.89, ion, transform = ax.transAxes, fontsize = 15, color = 'white')
        #ax.text(0.70,0.89, f"{(Ti,Tf)}", transform = ax.transAxes, fontsize = 15, color = 'white')
        plt.ylabel('Pixels', fontsize=12, labelpad = 5)
        plt.xlabel('Pixels', fontsize=12, labelpad = 5)
        ax.grid(which='major')
        ax.grid(which='minor', linewidth=0.3)
        ax.minorticks_on()
        
        textstr = '\n'.join((f"TOF-domain: {(Ti,Tf)}",f"Assignment: {ion}"))
        props = dict(boxstyle='round', facecolor='white', alpha=0.7)
        ax.text(0.05, 0.923, textstr, transform=ax.transAxes, fontsize=12, verticalalignment='center', bbox=props)
    
        
        if crosshairs:
            ax.axvline(162, c = 'w', ls = 'dashed', lw = 2, alpha = 0.75)
            ax.axhline(162, c = 'w', ls = 'dashed', lw = 2, alpha = 0.75)
            ax.add_patch(patches.Circle((162, 162), fill = False, color = 'white', ls = 'dashed', lw = 2))
            ax.add_patch(plt.Circle([162,162], 70, edgecolor='w', facecolor='none')) # for plotting circle at centre
            fig.savefig(f"ion_image_{i}_crosshairs.png")
            
        else:
            fig.savefig(f"ion_image_{i}_no_crosshairs.png")
    

In [ ]:
# 7. Obtain radial disributions of ions.

pixels = 324
rad_list_total = [] # for saving radial distributions as a list of lists

for i, (Ti, Tf) in enumerate(ToF_list):
    radius_for_dist = list(range(0, int(pixels/2)))
    intensity_for_dist = [0]*int(pixels/2) # list of zeros for intensities
    
    ion = ion_list[i]
    centre = centre_list[i]
    image = list_of_total_images[i]
    
    for ii in list(range(np.shape(image)[0])):
        for j in list(range(np.shape(image)[1])):
            intensity = image[ii,j]
            radius = int(np.sqrt((ii-162)**2+(j-162)**2))
            #print(radius)
            if 0 <= radius <= int(pixels/2):
                intensity_for_dist[radius-1] += intensity
            else:
                pass
    
    rad_list_total.append(intensity_for_dist)

    fig, ax = plt.subplots(figsize=(15, 6))
    beautify(ax)
    plt.plot(radius_for_dist,intensity_for_dist,color='black')
    ax.set_title('Radial distribution: ' + f"{ion} {Ti,Tf}", fontsize = 16)
    plt.xlabel('Radius / pixels', fontsize=14)
    plt.tick_params(axis='x', labelsize=12)
    plt.ylabel('Intensity / a.u.', fontsize=14)
    plt.tick_params(axis='y', labelsize=12)
    ax.set_ylim(0,1.10*max(intensity_for_dist))
    ax.grid(which='major')
    ax.grid(which='minor', linewidth=0.3)
    ax.minorticks_on()
    plt.show()
    #fig.savefig(f"radial_distribution_{i}.png")
    

In [ ]:
# 8. Compress data - not used (at this point in time) for real data.
#    Adapted from: "Recoil-frame covariance (for Part IIs, 220930).ipynb"

# Simulated data
if file_type == 0:
    compress_fac = 1 # this controls binning
    compress_df = df.copy()
    compress_df['x'] = (compress_df['x']/compress_fac).astype('int64')
    compress_df['y'] = (compress_df['y']/compress_fac).astype('int64')
    compress_centre_list = ((np.array(centre_list)/compress_fac).astype('int64'))
    print(compress_df)
    
# Real data
# if file_type == 1:
    
    #filtered_df_list = []
    compress_fac = 1 # this controls binning
    compress_centre_list = ((np.array(centre_list)/compress_fac).astype('int64'))
    
    #for df in zip(df_list):
        #df=df[0]
        #compress_df = df.filter(items=['x','y','ToF','tId']) # don't need delay column for static covariance analysis
        #compress_df = compress_df.loc[(compress_df['ToF'] >= 1000) & (compress_df['ToF'] <= 1350)] # filter over total TOF range where signal seen
        #filtered_df_list.append(compress_df)
        
    #compress_df = pd.concat(filtered_df_list)
        
#         compress_df = df.copy()
#         compress_df['x'] = (compress_df['x']/compress_fac).astype('int64')
#         compress_df['y'] = (compress_df['y']/compress_fac).astype('int64')
#         compress_centre_list = ((np.array(centre_list)/compress_fac).astype('int64'))
#         compress_df = df_total.copy()
#     compress_df = df_total.copy()
#     compress_df['x'] = (compress_df['x']/compress_fac).astype('int64')
#     compress_df['y'] = (compress_df['y']/compress_fac).astype('int64')

#     print(compress_df) # for total (summed over runs) concatenated dataframe

In [ ]:
# 9. Calculating ion covariances, calling the main covariance function defined in (2). Have to do data-frame by data-frame.
#    Adapted from: "Recoil-frame covariance (for Part IIs, 220930).ipynb"

compress_fac = 1
compress_centre_list = ((np.array(centre_list)/compress_fac).astype('int64'))

pixels = int(324/compress_fac)
#pixels = 162 # comment out if re-binning not needed
term_list = []
rad_term_list = []
count = 1

# fluorene 2023
# pixels = 324
# ToF_list = [(1613, 1616),(1617, 1621),(1631, 1634),(1643, 1646),(1650, 1652), (1657, 1662),(1662,1671),(1670, 1673), (1679,1683),(1688,1695),(1699,1705), (1708,1714), (1716,1723), (1725,1735),(1734,1741),(1743,1750), (1749,1755),(1756,1765),(1770,1776),(1783,1793), (1794,1803), (1810,1816),(1831,1841)]
# ion_list = ['N(++)','O(++)','C2(++)','O(+)','water(+)','carbon_dioxide(++)','C2(+)','dinitrogen(+)','dioxygen(+)','C3(+)','carbon_dioxide(+)','C4(+)','PAH(+++)','C5(+)','C11(++)','C6(+)','PAH(++)','C7(+)','C8(+)','C9(+)','C10(+)','C11(+)','PAH-H(+)']
# pq_list = [(2,2),(2,14),(14,2),(6,21),(21,6)] # just testing dication-dication atm
#pq_list = [(2,21),(21,2)]
# indices to consider (maximum = 22): 2,6,9,11,12,13,14,15,16,17,18,19,20,21

# triphenylene 2021
#ion_list = ['CH$^{2+}$','C$^{+}$','C$_{2}$H$_{2}^{+}$','C$_{3}$H$_{x}^{+}$','C$_{4}$H$_{x}^{+}$','C$_{5}$H$_{x}^{+}$','C$_{6}$H$_{x}^{+}$','C$_{7}$H$_{x}^{+}$','C$_{8}$H$_{x}^{+}$','C$_{9}$H$_{x}^{+}$']
#ToF_list = [(1044,1046),(1061,1063),(1097,1099),(1119,1121),(1139,1141),(1156,1158),(1172,1174),(1187,1189),(1202,1204),(1216,1218)]
#centre_list = [(165,162),(175,162),(178,162),(192,162),(195,162),(196,162),(198,162),(202,162),(205,162),(210,162)]

# for fluorene 2018 data
#pq_list = [(0,0),(0,1),(0,2),(0,3),(0,4),(0,5),(0,6),(0,7),(0,8),(0,9),(0,10),(1,1),(1,2),(1,3),(1,4),(1,5),(1,6),(1,7),(1,8),(1,9),(1,10),(2,2),(2,3),(2,4),(2,5),(2,6),(2,7),(2,8),(2,9),(2,10),(3,3),(3,4),(3,5),(3,6),(3,7),(3,8),(3,9),(3,10),(4,4),(4,5),(4,6),(4,7),(4,8),(4,9),(4,10),(5,5),(5,6),(5,7),(5,8),(5,9),(5,10),(6,6),(6,7),(6,8),(6,9),(6,10),(7,7),(7,8),(7,9),(7,10),(8,8),(8,9),(8,10),(9,9),(9,10),(10,10)] # this basically indexes the two ions in the ion list you want to calculate covariances for (where the first number is set as the reference ion)
#pq_list = [(0,0),(0,1),(0,2),(0,3),(0,4),(0,5),(0,6),(0,7),(0,8),(0,9),(1,1),(1,2),(1,3),(1,4),(1,5),(1,6),(1,7),(1,8),(2,2),(2,3),(2,4),(2,5),(2,6),(2,7),(3,3),(3,4),(3,5),(3,6),(4,4),(4,5),(5,5),(6,6),(7,7),(8,8),(9,9),(10,10)]

# for triphenylene 2021 data
#pq_list = [(0,1),(0,2),(0,3),(0,4),(0,5),(0,6),(0,7),(0,8),(0,9),(1,2),(1,3),(1,4),(1,5),(1,6),(1,7),(1,8),(1,9),(2,3),(2,4),(2,5),(2,6),(2,7),(2,8),(2,9),(3,4),(3,5),(3,6),(3,7),(3,8),(3,9),(4,5),(4,6),(4,7),(4,8),(4,9),(5,6),(5,7),(5,8),(5,9),(6,7),(6,8),(6,9),(7,8),(7,9),(8,9)]

# indene 2023
#ToF_list = [(1630,1634),(1665,1670),(1685,1697),(1700,1703),(1708,1718),(1720,1722),(1725,1735),(1740,1750),(1755,1765),(1785,1795)]
#ion_list = ['C1','C2','C3','C7(2+)','C4','IND(2+)','C5','C6','C7','IND(+)']
#pq_list = [(0,1),(0,2),(0,3),(0,4),(0,6),(0,7),(0,8),(1,2),(1,3),(1,4),(1,6),(1,7),(1,8),(2,4),(2,6),(2,7),(4,6),(0,0),(1,1),(2,2),(4,4)]

# fluorene 2023
# ToF_list = [(1831,1840),(1820,1830),(1810,1818),(1798,1802),(1780,1792),(1770,1778),(1755,1765),(1750,1752),(1742,1749),(1735,1740),(1725,1733),(1720,1722),(1707,1715),(1699,1704),(1685,1698),(1662,1672),(1630,1633)]
# ion_list = ['FLU(+)','C12(+)','C11(+)','C10(+)','C9(+)','C8(+)','C7(+)','FLU(++)','C6(+)','C11(++)','C5(+)','FLU(+++)/C9(++)','C4(+)','C3(+)','C7(++)/FLU(++++)','C2(+)','C1(+)']
# pq_list = [(16,1),(16,2),(15,2),(16,3),(15,3),(13,3),(16,4),(15,4),(13,4),(12,4),(16,5),(15,5),(13,5),(12,5),(10,5),(16,6),(15,6),(13,6),(12,6),(10,6),(8,6),(16,9),(15,9),(16,11),(15,11),(13,11),(12,11),(16,14),(15,14),(13,14),(12,14),(10,14),(8,14),(16,8),(15,8),(13,8),(12,8),(10,8),(12,10),(13,10),(15,10),(16,10),(13,12),(15,12),(16,12),(15,13),(16,13),(15,16),(16,16),(15,15),(13,13),(12,12),(10,10),(8,8)]

# CPP 2023
# ToF_list = [(1631,1636),(1665,1671),(1686,1694),(1701,1707),(1710,1715),(1718,1722),(1725,1733),(1736,1740),(1742,1749),(1752,1754),(1757,1765),(1766,1770),(1772,1778),(1780,1791),(1795,1805),(1806,1815),(1820,1828),(1830,1838),(1842,1850),(1851,1860),(1864,1869)]
# ion_list = ['C1','C2','C3','CPP(4+)','C4','C9(2+)','C5','C11(2+)','C6','C13(2+)','C7','CPP(2+)','C8','C9','C10','C11','C12','C13','C14','CPP(+)','[CPP-dinitrogen](+)']
# pq_list = [(0,1),(0,2),(0,4),(0,5),(0,6),(0,7),(0,8),(0,9),(0,10),(0,12),(0,13),(0,14),(0,15),(0,16),(0,17),(0,18),(1,2),(1,4),(1,5),(1,6),(1,7),(1,8),(1,9),(1,10),(1,12),(1,13),(1,14),(1,15),(1,16),(1,17),(2,4),(2,5),(2,6),(2,7),(2,8),(2,10),(2,12),(2,13),(2,14),(2,15),(2,16),(4,5),(4,6),(4,7),(4,8),(4,10),(4,12),(4,13),(4,14),(4,15),(5,6),(5,8),(6,8),(6,10),(6,12),(6,13),(6,14),(8,10),(8,12),(8,13),(10,12)]
#pq_list = [(4,7),(1,9),(8,5

# ethene sim data
ToF_list = [(7,7),(14,14)]
ion_list = ['$CH_{2}^{++}$','$CH_{2}^{+}$']
pq_list = [(0,0),(1,1),(0,1),(1,0)]
df_list = [df]

number_of_bins = 5 # for contingent covariance (in reality, number of bins is one minus from this value)

# for df in zip(df_list): # looping over each run number to sort energy bin stuff out. Comment out energy lines if not wanting to do contingent covariance or if Energy isn't in the df
        
#     df = df[0].sort_values(by=['Energy']) # load dataframe in ascending energy value
#     df_length = df.shape[0] # number of rows in total dataframe

#     min_energy = df['Energy'].min()
#     max_energy = df['Energy'].max()

#     FEL_energy_bins = []

#     index_range = np.arange(0, df_length, df_length/number_of_bins)
#     index_range_int = []

#     for a in range(len(index_range)): # need to make all values integers
#         index_range_int.append(int(index_range[a]))

#     print(f"Integer index range: {index_range_int}")

#     for index in index_range_int:
#         indexed_energy = df['Energy'].iloc[index]
#         FEL_energy_bins.append(indexed_energy)

#     print(f"FEL energy bin values: {FEL_energy_bins}")

for (p, q) in pq_list: # looping over each specified ion pair
    #     print(p,q)
    print('Covariance %s of %s.' % (count, len(pq_list)))

    (A_Ti, A_Tf), (B_Ti, B_Tf)  = ToF_list[p], ToF_list[q] # time windows for ion A and ion B, respectively
    A_centre, B_centre = compress_centre_list[p], compress_centre_list[q] # for simulated data, both are centred at (162,162)

        # ion A and ion B df filters
    #     B_mask = (((compress_df['ToF']>= B_Ti) & (compress_df['ToF'] <= B_Tf)))
    #     A_mask = (((compress_df['ToF']>= A_Ti) & (compress_df['ToF'] <= A_Tf)))
    
    Sij_store = []
    SiSj_store = []
    Sij_rad_store = []
    SiSj_rad_store = []
    
    for df in zip(df_list): # looping over each run number
        
        df = df[0]
        print(df)
        
        
#         for i in range(len(FEL_energy_bins)- 1): # looping over each energy bin; each bin should have approx. same number of shots in them. This is for PRE-T0
        
#             FEL_lower = FEL_energy_bins[i]
#             FEL_higher = FEL_energy_bins[i+1]

#             FEL_filtered_df = df[(df['Energy'] >= FEL_lower) & (df['Energy'] <= FEL_higher)]
#             #print(FEL_filtered_df)
#             compress_df=FEL_filtered_df

        FEL_filtered_df = df
        compress_df = FEL_filtered_df
        
        print(compress_df)
    
        B_mask = (compress_df['ToF'].values >= B_Ti) & (compress_df['ToF'].values <= B_Tf)
        A_mask = (compress_df['ToF'].values >= A_Ti) & (compress_df['ToF'].values <= A_Tf)

        df_A = compress_df[A_mask].copy()
        df_B = compress_df[B_mask].copy()

        print("df_A")
        print(df_A) # dataframe with just ion A in them
        print("df_B")
        print(df_B) # dataframe with just ion B in them

        df_A['x'] -= A_centre[1] # just centres x values
        df_A['y'] -= A_centre[0] # just centres y values
        df_A['r'] = np.sqrt((df_A['x'])**2 + (df_A['y'])**2) # caluculates the radius for each ion hit 
        df_A['theta'] = np.arctan2(df_A['x'], df_A['y']) + np.pi

        df_B['x'] -= B_centre[1] # just centres x values
        df_B['y'] -= B_centre[0] # just centres y values
        df_B['r'] = np.sqrt((df_B['x'])**2 + (df_B['y'])**2) # caluculates the radius for each ion hit 
        df_B['theta'] = np.arctan2(df_B['x'], df_B['y']) + np.pi

        print("df_A new")
        print(df_A)
        print("df_B new")
        print(df_B)

        shot_list = np.arange(np.min(FEL_filtered_df.tId),np.max(FEL_filtered_df.tId) + 1) # just an array of shot IDs
        #print("shot list")
        #print(shot_list)

    #     print(df_A.dtypes, df_B.dtypes, shot_list.dtype)
    #     print(df_B.dtypes)
        Sij, SiSj, Sij_rad, SiSj_rad = two_fold_recoil_frame_covariance(df_A, df_B, shot_list, pixels, norm = True) # MAIN COVARIANCE FUNCTION
        #count += 1
        Sij_store.append(Sij)
        SiSj_store.append(SiSj)
        Sij_rad_store.append(Sij_rad)
        SiSj_rad_store.append(SiSj_rad)
        
    # summing over each run number and energy bin
    Sij = sum(Sij_store)
    SiSj = sum(SiSj_store)
    Sij_rad = sum(Sij_rad_store)
    SiSj_rad = sum(SiSj_rad_store)
    
    term_list.append((Sij, SiSj))
    rad_term_list.append((Sij_rad,SiSj_rad))
    print('Finished covariance %s of %s.' % (count, len(pq_list)))
    count += 1
    #     break

print('Finished.')  

In [ ]:
# 10. Plotting the recoil-frame covariance maps.
#     Adapted from: "Recoil-frame covariance (for Part IIs, 220930).ipynb"

# fluorene 2018
#pq_list = [(0,0),(0,1),(0,2),(0,3),(0,4),(0,5),(0,6),(0,7),(0,8),(0,9),(1,1),(1,2),(1,3),(1,4),(1,5),(1,6),(1,7),(1,8),(2,2),(2,3),(2,4),(2,5),(2,6),(2,7),(3,3),(3,4),(3,5),(3,6),(4,4),(4,5),(5,5),(6,6),(7,7),(8,8),(9,9),(10,10)]
#print(pq_list)

# triphenylene 2021
#pq_list = [(0,1),(0,2),(0,3),(0,4),(0,5),(0,6),(0,7),(0,8),(0,9),(1,2),(1,3),(1,4),(1,5),(1,6),(1,7),(1,8),(1,9),(2,3),(2,4),(2,5),(2,6),(2,7),(2,8),(2,9),(3,4),(3,5),(3,6),(3,7),(3,8),(3,9),(4,5),(4,6),(4,7),(4,8),(4,9),(5,6),(5,7),(5,8),(5,9),(6,7),(6,8),(6,9),(7,8),(7,9),(8,9)]
#print(pq_list)

# fluorene 2023
#ion_list = ['FLU(+)','C12(+)','C11(+)','C10(+)','C9(+)','C8(+)','C7(+)','FLU(++)','C6(+)','C11(++)','C5(+)','[FLU(+++)/C9(++)]','C4(+)','C3(+)','C7(++)','C2(+)','C1(+)']

for i, (Sij, SiSj) in enumerate(term_list):
    pq_tuple = pq_list[i]
    # print(pq_tuple)
    
    # <XY>
#     fig, ax = plt.subplots(figsize=(6, 6))
#     maximage = Sij.max()
#     picture = ax.imshow(Sij, interpolation='nearest', cmap='inferno', vmin=0, vmax=0.8*maximage)
#     ax.set_title(str(ion_list[p]) + ' / ' + str(ion_list[q]) + ': <XY>')

    # <X><Y>
#     fig2, ax2 = plt.subplots(figsize=(6, 6))
#     maximage2 = SiSj.max()
#     picture = ax2.imshow(SiSj, interpolation='nearest', cmap='inferno', vmin=0, vmax=maximage2)
#     ax2.set_title(str(ion_list[p]) + ' / ' + str(ion_list[q]) + ': <X><Y>')

    # <XY>-<X><Y>
    fig3, ax3 = plt.subplots(figsize=(8, 8)) # recoil-frame covariance plot - just one component of covariance
    cov = Sij-SiSj
    maximage3 = cov.max()
    print(maximage3)
    print(cov.min())
    #print(maximage3)
    if pq_tuple[0] != pq_tuple[1]: # if not looking at autocovariance
        picture = ax3.imshow(Sij-SiSj, interpolation='nearest', cmap='inferno', vmin=0, vmax=maximage3)
        plt.arrow(270,162,20,0, color='w',head_width=5)
    elif pq_tuple[0] == pq_tuple[1]: # if looking at autocovariance (for image oversaturation to bring out covariance features against an overwhelming variance signal)
        picture = ax3.imshow(Sij-SiSj, interpolation='nearest', cmap='inferno', vmin=0, vmax=maximage3)
    ax3.set_title(str(ion_list[pq_tuple[0]]) + ' / ' + str(ion_list[pq_tuple[1]]) + ' recoil-frame covariance map (static)')
    plt.xlabel('Pixels', fontsize = 12, labelpad = 5)
    plt.ylabel('Pixels', fontsize = 12, labelpad = 5)
    cbar = plt.colorbar(picture, shrink=0.82)
    cbar.set_label('Covariance / a.u. ', rotation=270, fontsize = 12,labelpad=20)
    plt.show()
    
    fig3.savefig(f"{pq_tuple[0]}_{pq_tuple[1]}_recoil_frame_covariance_static.png",bbox_inches="tight")
    #np.save(f"{ion_list[pq_tuple[0]]}_{ion_list[pq_tuple[1]]}_covar_slice.npy",cov) # save rf maps as np arrays

In [ ]:
# 13. Scaling the radial covariance matrices relative to the maximum matrix entry in a given matrix, and re-plotting. There is
#     also an option to scale the matrices relative to one another. Re-bins into 81x81 maps, reinflates to 162x162, and plots.

radcov_matrix_maxvals = []
radcov_rebinned_list = []
radcov_list = []

for i, (Sij_rad, SiSj_rad) in enumerate(rad_term_list):
    pq_tuple = pq_list[i]
    #if pq_tuple[0] != pq_tuple[1]: # not looking at autocorrelation:
    radcov = Sij_rad - SiSj_rad
        
    radcov_list.append(radcov)
        
        # re-binning into 81x81, then reinflating for plotting
        #radcovmean = np.mean(np.split(np.mean(np.split(radcov, 162 // 2, axis=1), axis=-1), 162 // 2, axis=1), axis=-1) # for binning; 81 x 81 matrix
        #radcov = np.kron(radcovmean, np.ones((2, 2))) # reinflates re-binned matrix for plotting
        #radcov_rebinned_list.append(radcov)
        
    radcov_matrix_maxvals.append(np.max(radcov))
        
scaling_factor = max(radcov_matrix_maxvals) # maximum matrix entry across all radial covariance plots
print("Scaling factor: " + str(scaling_factor))
radcov_matrix_maxvals_norm = radcov_matrix_maxvals/scaling_factor
print("To check for correct normalisation (should equal 1): " + str(max(radcov_matrix_maxvals_norm)))

for i, radcov in enumerate(radcov_list):
    
    pq_tuple = pq_list[i]
    
    #if pq_tuple[0] != pq_tuple[1]: # not looking at autocorrelation:
        
        #radcov /= scaling_factor
        
    maximage = radcov.max()
    print(maximage)
    fig = plt.figure(figsize=(12, 12))
    gs = fig.add_gridspec(2, 2,  width_ratios=(3, 1), height_ratios=(1, 3),
                          left=0.1, right=0.9, bottom=0.1, top=0.9,
                          wspace=0, hspace=0)
    ax = fig.add_subplot(gs[1, 0])

    #ax.set_title(str(ion_list[pq_tuple[0]]) + '/' + str(ion_list[pq_tuple[1]]) + ': <XY> - <X><Y>', fontsize = 16)
    ax.set_xlabel(str(ion_list[pq_tuple[1]]) + ' ion-image radius / pixels', fontsize=14, labelpad=15)    
    ax.set_ylabel(str(ion_list[pq_tuple[0]]) + ' ion-image radius / pixels', fontsize=14, labelpad=15)
    ax.grid(which='major')
    ax.grid(which='minor', linewidth=0.3)
    ax.minorticks_on()
    ax_x = fig.add_subplot(gs[0, 0], sharex=ax) # radial dist (top row)
    ax_y = fig.add_subplot(gs[1, 1], sharey=ax) # radial dist (column)
    picture = ax.imshow(radcov, interpolation='nearest', cmap='inferno', vmin=0, vmax=0.5*maximage) # edit vmax as necessary

    ax_x.plot(radius_for_dist, rad_list_total[pq_tuple[1]], color = 'purple')
    ax_y.plot(rad_list_total[pq_tuple[0]], radius_for_dist, color = 'purple')
    ax_x.set_ylabel('Intensity / a.u.', fontsize=14,labelpad=15)
    ax_y.set_xlabel('Intensity / a.u.', fontsize=14,labelpad=15)
    ax_x.tick_params(axis="x",direction="out", pad=5)
    ax_y.tick_params(axis="y",direction="out", pad=5)
    plt.setp(ax_x.get_xticklabels(), visible=False)
    plt.setp(ax_y.get_yticklabels(), visible=False)
    ax_x.grid(which='major')
    ax_x.grid(which='minor', linewidth=0.3)
    ax_x.minorticks_on()
    ax_y.grid(which='major')
    ax_y.grid(which='minor', linewidth=0.3)
    ax_y.minorticks_on()

    # colorbar formatting
    fig.subplots_adjust(right=0.8)
    cbar_ax = fig.add_axes([1, 0.15, 0.05, 0.7])
    fig.colorbar(picture, cax=cbar_ax)
    cbar_ax.set_ylabel('Covariance / a.u.', fontsize = 14, rotation = 270, labelpad=25)


    ax_x.text(0.95, 0.82, str(ion_list[pq_tuple[1]]), horizontalalignment='right', verticalalignment='bottom', transform=ax_x.transAxes, fontsize = 14, weight='bold')
    ax_y.text(0.90, 0.03, str(ion_list[pq_tuple[0]]), horizontalalignment='right', verticalalignment='bottom', transform=ax_y.transAxes, fontsize = 14, weight='bold')

    fig.savefig(str(i) + '_radial_covariance_scaled.png',bbox_inches="tight")